In [1]:
from pcraster import *
from pcraster.framework import *
import matplotlib.pyplot as plt
import numpy as np
import random 

In [5]:
shrubDevelopment = 0

class ShrubEncroachment(DynamicModel):

  def __init__(self, bR, gR, rR):
      DynamicModel.__init__(self)
      setclone('clone200.map')
      self.burningRate = bR
      self.grazingRate = gR
      self.removingRate = rR

In [4]:
 def initial(self):
      # shrub middle 10%, other area 10% empty, 80% grass
      self.resultMap = self.readmap("initialDividedNominal")
      self.Results(self.resultMap, "Results/result_inital")
      self.range = 40
      self.totalCells = 200 * 200
      self.currentTimestep = 0

In [14]:
def dynamic(self):
      self.currentTimestep = self.currentTimestep +1 
      global shrubDevelopment
      #0: Empty
      #1: grass
      #2: Shrub
      #3: Burning

      estab_rate_shrub_on_empty = 6.8+mapnormal()*2.323
      estab_rate_shrub_on_grass = 0.387+mapnormal()*0.082
      estab_rate_shrub_on_burned = 38.8+mapnormal()*9.768
      repro_rate_shrub_on_empty = 0.109+mapnormal()*0.01
      repro_rate_shrub_on_grass = 0.061+mapnormal()*0.016
      clonal_rate_gras_on_empty = 0.5
      competition_shrub_center = 0.00015+mapnormal()*0.001
      mortality_rate_shrub = 0.028+mapnormal()*0.0005
      mortality_rate_grass = 0.125
      estab_rate_grass_on_empty = 0.8
      estab_rate_burned_on_shrub =38.8 + mapnormal()*9.768

      self.resultMap = ifthenelse(self.resultMap == 3, 0, self.resultMap)
      total_empty_cells = maptotal(scalar(self.resultMap == 0))
      total_grass_cells = maptotal(scalar(self.resultMap == 1))
      total_shrub_cells = maptotal(scalar(self.resultMap == 2))
      numberOfBurnedCells = maptotal(scalar(self.resultMap == 3))

      global_prob_empty = total_empty_cells / self.totalCells #first step 0.1
      global_prob_grass = total_grass_cells / self.totalCells #first step 0.8
      global_prob_shrub = total_shrub_cells / self.totalCells # first step 0.1

      self.Results(total_empty_cells, "Results/eCells")
      self.Results(global_prob_empty, "Results/globalE")
      self.Results(global_prob_grass, "Results/globalG")
      self.Results(global_prob_shrub, "Results/globalS")
      
      local_prob_qSG=scalar(ifthen(self.resultMap == 2, window4total(scalar(self.resultMap == 1))/4))
      local_prob_qSS=scalar(ifthen(self.resultMap == 2, window4total(scalar(self.resultMap == 2))/4))
      local_prob_qSE=scalar(ifthen(self.resultMap == 2, window4total(scalar(self.resultMap == 0))/4))
      local_prob_qSB=scalar(ifthen(self.resultMap == 2, window4total(scalar(self.resultMap == 3))/4))
      
      local_prob_qGB=scalar(ifthen(self.resultMap == 1, window4total(scalar(self.resultMap == 3))/4))
      local_prob_qGE=scalar(ifthen(self.resultMap == 1, window4total(scalar(self.resultMap == 0))/4))
      local_prob_qGS=scalar(ifthen(self.resultMap == 1, window4total(scalar(self.resultMap == 2))/4))

      local_prob_qEG=scalar(ifthen(self.resultMap == 0, window4total(scalar(self.resultMap == 1))/4))
      local_prob_qES=scalar(ifthen(self.resultMap == 0, window4total(scalar(self.resultMap == 2))/4))

      local_prob_qBS=scalar(ifthen(self.resultMap == 3, window4total(scalar(self.resultMap == 2))/4))
      local_prob_qBG=scalar(ifthen(self.resultMap == 3, window4total(scalar(self.resultMap == 1))/4))


      self.Results(local_prob_qSG, "Results/qSG")
      self.Results(local_prob_qSS, "Results/qSS")
      self.Results(local_prob_qGE, "Results/qGE")
      self.Results(local_prob_qSE, "Results/qSE")
      self.Results(local_prob_qSB, "Results/qSB")
      self.Results(local_prob_qGB, "Results/qGB")

      #Transition Rules

      # 1 Colonization of empty cells by shrub
      transition_empty_shrub = ((1-local_prob_qES) * estab_rate_shrub_on_empty + repro_rate_shrub_on_empty) * local_prob_qES*(1-self.grazingRate)
      #transition_empty_shrub = 0.5
      
      # 2 Colonization of empty cells by grass
      transition_empty_grass = clonal_rate_gras_on_empty * local_prob_qEG + estab_rate_grass_on_empty * global_prob_grass
      #transition_empty_grass = 0.5
      
      # 3 Colonization of grass cells by shrub
      transition_grass_shrub = ((1-local_prob_qGS) *estab_rate_shrub_on_grass + repro_rate_shrub_on_grass) * local_prob_qGS*(1-self.grazingRate)
      #transition_grass_shrub = 0.5
      
      # 4 Mortality
      mortality_shrub = mortality_rate_shrub + competition_shrub_center * local_prob_qSS
      mortality_grass = mortality_rate_grass
      
      # 5 Colonization of burned cells by shrub
      transition_burned_shrub = ((1-local_prob_qBS) * estab_rate_shrub_on_burned + repro_rate_shrub_on_empty) * local_prob_qBS*(1-self.grazingRate)
      
      # 6 Colonization of burned cells by grass
      transition_burned_grass = clonal_rate_gras_on_empty * local_prob_qBG + estab_rate_grass_on_empty * global_prob_grass
      #transition_burned_grass = 0.5

